 # Training Model
 Keras on Tensorflow


In [1]:
import os, sys
import cv2
import numpy as np
import uuid
import tensorflow as tf
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools
import json
from pprint import pprint

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.models import UNet, UNet_hp
from core.metrics import iou_coef, dice_coef

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorboard.plugins.hparams import api as hp
from tensorflow.summary import create_file_writer

from datetime import datetime

from IPython import get_ipython
# %load_ext autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
# %autoreload 2
get_ipython().run_line_magic('autoreload', '2')
# %load_ext tensorboard
get_ipython().run_line_magic('load_ext', 'tensorboard')

In [2]:
from packaging import version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

 ## Load Training Dataset

In [4]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')

# input set
crop_input_set = '2019_12_06_17_06'
# crop_input_set = '2019_12_10_15_33_5x' # small training set

imginput = os.path.join('dl_seg_project_raw', 'data_crop', crop_input_set,)
imgpath = os.path.join(path, imginput)

print(imgpath)

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')
print(img_dir)
print(label_dir)

# check if the output folder exist. If not, create a folder
dir_checker('logs', path)
path_logs = os.path.join(path, 'logs')
dir_checker('fit', path_logs)
dir_checker('model', path_logs)
dir_checker('pars', path_logs)

Load Images...
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\images
D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\labels
logs exists in D:PerlmutterData
fit exists in D:PerlmutterData\logs
model exists in D:PerlmutterData\logs
pars exists in D:PerlmutterData\logs


 Print the first file name.

In [5]:
imgpath_all = list(paths.list_images(imgpath))
print(imgpath_all[0])

D:PerlmutterData\dl_seg_project_raw\data_crop\2019_12_06_17_06\images\autophagosome\0001.tif


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [6]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
date =  datetime.now().strftime("%Y_%m_%d")
seed = 100
batch_size = 16
epoch = 100
validation_steps = 20
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = None
IMG_WIDTH = None
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[1]]
learning_rate = 1e-5
loss = "binary_crossentropy"
metrics = ['accuracy', iou_coef, dice_coef]

metrics_name = []
for f in metrics:
    if callable(f):
        metrics_name.append(f.__name__)
    else:
        metrics_name.append(f)

In [7]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

# create parameter
pars = dict(
                # basic information
                timestamp = timestamp,
                date = date,
                seed = seed,
                batch_size = batch_size,
                
                # Data generator
                crop_input_set = crop_input_set,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                
                # training class
                classes = classes,
                inputclass = inputclass,
    
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                
                # Build model
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                epoch = epoch, 
                loss = loss,
                metrics_name = metrics_name,
                learning_rate = learning_rate,
                )

# save parameter
path_pars = os.path.join(path_logs, 'pars')
dir_checker(inputclass[0], path_pars)
dir_checker(date, os.path.join(path_pars, inputclass[0]))

pprint(pars)

nucleus exists in D:PerlmutterData\logs\pars
2019_12_13 exists in D:PerlmutterData\logs\pars\nucleus
{'IMG_HEIGHT': None,
 'IMG_WIDTH': None,
 'batch_size': 16,
 'classes': ['cell_membrane', 'nucleus', 'autophagosome'],
 'crop_input_set': '2019_12_06_17_06',
 'data_gen_img_args': {'height_shift_range': 0.1,
                       'horizontal_flip': True,
                       'rescale': 0.00392156862745098,
                       'rotation_range': 90.0,
                       'shear_range': 0.07,
                       'validation_split': 0.1,
                       'vertical_flip': True,
                       'width_shift_range': 0.1,
                       'zoom_range': 0.2},
 'data_gen_label_args': {'height_shift_range': 0.1,
                         'horizontal_flip': True,
                         'rescale': 0.00392156862745098,
                         'rotation_range': 90.0,
                         'shear_range': 0.07,
                         'validation_split': 0.1,
       

In [8]:
par_file_dir = os.path.join(path_pars, inputclass[0], date, 'pars_' + timestamp + '.json')
print(par_file_dir)

with open(par_file_dir, 'w') as outfile:
    json.dump(pars, outfile, indent=4)


D:PerlmutterData\logs\pars\nucleus\2019_12_13\pars_2019_12_13_17_28.json


In [9]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [10]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)

Found 10395 images belonging to 1 classes.
Found 10395 images belonging to 1 classes.
Found 1155 images belonging to 1 classes.
Found 1155 images belonging to 1 classes.


In [11]:

# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)


In [12]:
'''
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)
'''

'\ntrain_generator = zip(train_image_generator, train_label_generator)\nvalid_generator = zip(valid_image_generator, valid_label_generator)\n'

 ## Training

In [13]:
print("Start training...")

Start training...


 ### Define Callbacks

 ### Setup the model

In [14]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1 - validation_split) // batch_size
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 3043.0


In [15]:
'''
logdir_tb = os.path.join(path_logs, 'fit', inputclass[0], date)
ip= 'localhost'
port = '6006'
'''

"\nlogdir_tb = os.path.join(path_logs, 'fit', inputclass[0], date)\nip= 'localhost'\nport = '6006'\n"

In [16]:
'''
%tensorboard --logdir {logdir_tb} --host {ip}
# get_ipython().run_line_magic('tensorboard', '--logdir', )
'''

"\n%tensorboard --logdir {logdir_tb} --host {ip}\n# get_ipython().run_line_magic('tensorboard', '--logdir', )\n"

In [17]:
dir_checker(inputclass[0], os.path.join(path_logs, 'fit'))
dir_checker(date, os.path.join(path_logs, 'fit', inputclass[0]))
dir_checker(inputclass[0], os.path.join(path_logs, 'model'))
dir_checker(date, os.path.join(path_logs, 'model', inputclass[0]))

nucleus exists in D:PerlmutterData\logs\fit
2019_12_13 exists in D:PerlmutterData\logs\fit\nucleus
nucleus exists in D:PerlmutterData\logs\model
2019_12_13 exists in D:PerlmutterData\logs\model\nucleus


In [18]:
logdir = os.path.join(path_logs, 'fit', inputclass[0], date, timestamp)

# Hyperparameter
hparam_tuning_path = os.path.join(logdir, 'hyparam_tuning')

HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.5, 0.7))
hparams_list = [HP_DROPOUT]

with tf.summary.create_file_writer(hparam_tuning_path).as_default():
    hp.hparams_config(
    hparams=[HP_DROPOUT],
    metrics=[hp.Metric('accuracy', display_name='Accuracy'), 
             hp.Metric('iou_coef', display_name='IoU_Coef'),
             hp.Metric('dice_coef', display_name='Dice_Coef')],
  )
    

In [19]:
def run(run_name, path_logs, hparams):
    
    run_dir = os.path.join(hparam_tuning_path, run_name)
    
    # checkpoint
    modelfilename = 'model_' + timestamp + '.h5'
    dir_checker(timestamp, os.path.join(path_logs, 'model', inputclass[0], date))
    dir_checker(run_name, os.path.join(path_logs, 'model', inputclass[0], date, timestamp))
    modelfile_path = os.path.join(path_logs, 'model', inputclass[0], date, timestamp, run_name, modelfilename)

    checkpointer = ModelCheckpoint(filepath = modelfile_path, 
                                   monitor = 'val_loss', 
                                   mode = 'min', 
                                   verbose = 1, 
                                   save_best_only = True)

    # early stopping 
    early_stopping = EarlyStopping(monitor='val_loss',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4)

    # learning rate adjustment
    reduceLR = ReduceLROnPlateau(monitor='val_loss',
                        factor=0.1,
                        patience=4,
                        verbose=1,
                        min_delta=1e-4)

    # tensorboard
    
    file_writer = create_file_writer(os.path.join(path_logs, 'fit', inputclass[0], date, timestamp, "metrics"))
    file_writer.set_as_default()

    metrics = ['accuracy', iou_coef, dice_coef]
    
    tensorboard_callback = TensorBoard(log_dir = logdir, 
                                       profile_batch = 0, 
                                       update_freq= 500,
                                       histogram_freq = 1
                                       )

    # compile callbacks
    callbacks = [checkpointer, tensorboard_callback, early_stopping, reduceLR]
    
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        
        # prepare the model
        unetmodel = UNet_hp(shape = (IMG_HEIGHT, IMG_WIDTH), 
                         lr = learning_rate, 
                         loss = loss,
                         metrics = metrics,
                         hparams = hparams,
                            hparams_list = hparams_list, 
                        )
        
        # train the model
        unetmodel.fit_generator(
                            generator = train_generator, 
                            validation_data = valid_generator,
                            validation_steps = validation_steps,
                            steps_per_epoch = steps_per_epoch,
                            epochs = epoch,  
                            callbacks = callbacks,
                            verbose = 1, 
                            )
        _, accuracy, iou, dice,  = unetmodel.evaluate_generator(valid_generator, steps = 50, verbose=1)
        tf.summary.scalar('accuracy', accuracy, step = 1)
        tf.summary.scalar('iou_coef', iou, step = 1)
        tf.summary.scalar('dice_coef', dice, step = 1)

In [20]:
session_num = 0

for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    hparams = {
      HP_DROPOUT: dropout_rate,
    }
    print(hparams)
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    
    run(run_name, path_logs, hparams)
    session_num += 1


{HParam(name='dropout', domain=RealInterval(0.5, 0.7), display_name=None, description=None): 0.5}
--- Starting trial: run-0
{'dropout': 0.5}
2019_12_13_17_28 does not exist in D:PerlmutterData\logs\model\nucleus\2019_12_13
run-0 does not exist in D:PerlmutterData\logs\model\nucleus\2019_12_13\2019_12_13_17_28
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 6 36928       conv2d[0][0] 

Epoch 1/100
3042/3043 [============================>.] - ETA: 0s - loss: 0.1182 - accuracy: 0.9508 - iou_coef: 0.3121 - dice_coef: 0.3382
Epoch 00001: val_loss improved from inf to 1.42507, saving model to D:PerlmutterData\logs\model\nucleus\2019_12_13\2019_12_13_17_28\run-0\model_2019_12_13_17_28.h5
3043/3043 [==============================] - 3027s 995ms/step - loss: 0.1182 - accuracy: 0.9508 - iou_coef: 0.3121 - dice_coef: 0.3382 - val_loss: 1.4251 - val_accuracy: 0.6888 - val_iou_coef: 0.0636 - val_dice_coef: 0.0881
Epoch 2/100
3042/3043 [============================>.] - ETA: 0s - loss: 0.0361 - accuracy: 0.9868 - iou_coef: 0.3836 - dice_coef: 0.3995
Epoch 00002: val_loss did not improve from 1.42507
3043/3043 [==============================] - 3273s 1s/step - loss: 0.0361 - accuracy: 0.9868 - iou_coef: 0.3836 - dice_coef: 0.3995 - val_loss: 1.8252 - val_accuracy: 0.7204 - val_iou_coef: 0.1257 - val_dice_coef: 0.1530
Epoch 3/100
3042/3043 [============================>.] - ETA: 0s

Epoch 1/100
3042/3043 [============================>.] - ETA: 0s - loss: 0.1431 - accuracy: 0.9401 - iou_coef: 0.2939 - dice_coef: 0.3252
Epoch 00001: val_loss improved from inf to 0.58333, saving model to D:PerlmutterData\logs\model\nucleus\2019_12_13\2019_12_13_17_28\run-1\model_2019_12_13_17_28.h5
3043/3043 [==============================] - 3002s 987ms/step - loss: 0.1430 - accuracy: 0.9402 - iou_coef: 0.2939 - dice_coef: 0.3253 - val_loss: 0.5833 - val_accuracy: 0.7634 - val_iou_coef: 0.1442 - val_dice_coef: 0.2080
Epoch 2/100
3042/3043 [============================>.] - ETA: 0s - loss: 0.0446 - accuracy: 0.9838 - iou_coef: 0.3754 - dice_coef: 0.3931
Epoch 00002: val_loss improved from 0.58333 to 0.31961, saving model to D:PerlmutterData\logs\model\nucleus\2019_12_13\2019_12_13_17_28\run-1\model_2019_12_13_17_28.h5
3043/3043 [==============================] - 3003s 987ms/step - loss: 0.0446 - accuracy: 0.9838 - iou_coef: 0.3754 - dice_coef: 0.3931 - val_loss: 0.3196 - val_accuracy

!nvidia-smi